# Camera calibration Sim

In [ ]:
# Import librarys
import time
import math
import numpy as np
import cv2
import random
import glob
import matplotlib.pyplot as plt
from scipy.linalg import svd
from coppeliasim_zmqremoteapi_client import RemoteAPIClient

## CoppeliaSim Scene config


In [ ]:

# Connect and configure the simulation 
client = RemoteAPIClient()
sim = client.getObject('sim')

# When simulation is not running, ZMQ message handling could be a bit
# slow, since the idle loop runs at 8 Hz by default. So let's make
# sure that the idle loop runs at full speed for this program:
defaultIdleFps = sim.getInt32Param(sim.intparam_idle_fps)   
sim.setInt32Param(sim.intparam_idle_fps, 0)

# Get the vision sensor handle
visionSensorHandle = sim.getObject('/Vision_sensor')
cubo = sim.getObject('/Cuboid[3]')

## Recover camera parameters from scene

In [ ]:
isZhang = False

# Vision sensor internal parameters
# Run this before to adjust the perspective angle
def get_perspective_angle(focal_length, sensor_size):
    perspective_angle = 2*math.atan(sensor_size / (2*focal_length))
    perspective_angle = perspective_angle*(180/math.pi)
    print(perspective_angle)
    return perspective_angle

get_perspective_angle(3.04,3.68)

focal_length = 3.04
# Defined in vision sensor parameters
image_size = (1280,720)

# Pixel density defined by the sensor resolution
pixel_density = (830,829)

# Cria a matriz K
k_matrix = np.array([[focal_length*pixel_density[0], 0, image_size[0]/2],
                    [0, focal_length*pixel_density[1], image_size[1/2]],
                    [0, 0, 1]], dtype=np.float64)

# Start simulation in CoppeliaSim
sim.startSimulation()

# Get Vision_sensor transformation matrix
transformation_matrix = sim.getObjectMatrix(visionSensorHandle, -1)

sim.stopSimulation()

# Transforma a transformation_matrix em numpy

# Cria a matriz de câmera C

## Get the calibration images

In [ ]:
# Start simulation in CoppeliaSim
sim.startSimulation()

# index of the calibration image
n = 1

# See the Vision sensor image
while (t := sim.getSimulationTime()) < 10:
    img, resX, resY = sim.getVisionSensorCharImage(visionSensorHandle)
    img = np.frombuffer(img, dtype=np.uint8).reshape(resY, resX, 3)
    # In CoppeliaSim images are left to right (x-axis), and bottom to top (y-axis)
    # (consistent with the axes of vision sensors, pointing Z outwards, Y up)
    # and color format is RGB triplets, whereas OpenCV uses BGR:
    img = cv2.flip(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), 0)
    #cv2.imshow('', img)
    #cv2.waitKey(1)

    p=sim.getObjectPosition(cubo,-1)

    print(p, transformation_matrix,'\n','\n')
    time.sleep(0.75)

    cv2.imwrite(f'image{n}.jpg',img)

sim.stopSimulation()

cv2.destroyAllWindows()

sim.stopSimulation()

## Extract world points

In [ ]:
if isZhang: 
    patternSize = (7,7)
    squareSize = (10)
    imgSize = (1280,720)

    def construct3DPoints(patternSize,squareSize):
        X = np.zeros((patternSize[0]*patternSize[1],3), np.float32)
        X[:,:2] = np.mgrid[0:patternSize[0],0:patternSize[1]].T.reshape(-1,2)
        X = X * squareSize
        return X

    boardPoints = construct3DPoints(patternSize,squareSize)
    worldPoints = []

else: 
    worldPoints = [] 

## Extract image points

In [ ]:
if isZhang:
    counter = 0
    imagePoints = []
    images = glob.glob('../calibration-images/*.jpg')

    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCornersSB(gray, patternSize, None)
        if ret == True:
            print("Corners found in image" + str(fname)) #- see if corners are found 
            imagePoints.append(corners)
            worldPoints.append(boardPoints)
            counter+=1

    print("using " + str(counter) + " images")

else: 
    img = cv2.imread('image.jpg') 
  
    # convert image to gray scale image 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    
    # detect corners with the goodFeaturesToTrack function. 
    corners = cv2.goodFeaturesToTrack(gray, 27, 0.01, 10) 
    corners = np.int0(corners) 

    print(corners)
    
    # we iterate through each corner,  
    # making a circle at each point that we think is a corner. 
    for i in corners: 
        x, y = i.ravel() 
        cv2.circle(img, (x, y), 3, 255, -1) 
    
    plt.imshow(img), plt.show() 

## Compute the camera matrix

In [ ]:
if isZhang: 
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    flagsCalib = cv2.CALIB_RATIONAL_MODEL
    ret, cameraMatrix, k, rvecs, tvecs = cv2.calibrateCamera(worldPoints, imagePoints, imgSize, None, None,flags=flagsCalib)

    print("Using "+str(counter)+" of "+str(len(images))+" images")
    print("RMS re-projection error:", ret)
    print("Camera Matrix:\n", cameraMatrix)
    print("Distortion Parameters:\n", k)
    
else: 

    def calibrate_camera(world_points, image_points):
        """
        Função para encontrar a matriz de câmera C usando o algoritmo DLT.
        
        Args: 
            world_points: Array de pontos 3D na cena.
            image_points: Array de pontos 2D projetados na imagem.

        Return: 
            C_matrix: numpy array contendo a matriz de câmera estimada. 
        """

        if len(world_points) != len(image_points): 
            raise ValueError("Must be the same number of World points and Image points")
        
        if len(world_points) < 6 | len(image_points) < 6 :
            raise ValueError("Must have a minimum of 6 points to compute the camera matrix")


        Q = []
        for i in range(len(world_points)):
            X, Y, Z = world_points[i]
            u,v  = image_points[i]
            Q.append([X, Y, Z, 1, 0, 0, 0, 0, -u*X, -u*Y, -u*Z, -u])
            Q.append([0, 0, 0, 0, X, Y, Z, 1, -v*X, -v*Y, -v*Z, -v])

        Q = np.array(Q)
        #print(Q)

        # Resolva o sistema de equações usando SVD
        _, _, V = svd(Q)
        c = V[-1, :12]
        #print(c)

        # Reconstrua a matriz de projeção da câmera
        C_matrix = c.reshape(3, 4)
        print(C_matrix)

        return C_matrix
    
    camera_matrix = calibrate_camera(worldPoints,imagePoints)

## Results 

In [ ]:
# Plots and stuff like that